# エピソディックメモリを使用したデバッグアシスタント

## 概要

このノートブックでは、リフレクション付きの **AgentCore エピソディックメモリ**を使用して、インテリジェントな**デバッグアシスタント**を構築する方法を示します。エージェントは過去のデバッグセッションから学習し、履歴の経験に基づいてコンテキスト対応のガイダンスを提供します。

### エピソディックメモリとは?

**エピソディックメモリ**は、構造化されたコンテキストを持つ完全なやり取りシーケンス（エピソード）をキャプチャします。孤立した事実を保存するセマンティックメモリとは異なり、エピソディックメモリは以下を保持します:
- **完全な会話フロー**: 問題の説明から解決までの完全なデバッグセッション
- **時間的コンテキスト**: 取られたアクションのシーケンスとタイミング
- **結果**: デバッグの試行が成功したか失敗したか
- **構造化されたターン**: 思考、アクション、観察を含む個々のステップ

![エピソディックメモリ](./episodic_memory.png)

### リフレクションとは?

**リフレクション**は、複数のエピソードから自動的に抽出された合成インサイトです。以下を提供します:
- **パターン認識**: 類似したエピソード間で共通の問題とその解決策
- **ベストプラクティス**: 成功したデバッグセッションでうまくいった戦略
- **よくある落とし穴**: 失敗した試行に基づいて避けるべきミス
- **戦略的ガイダンス**: 類似の問題にアプローチするための高レベルのアドバイス

**出力構造:**
- **エピソード**: `debugging/{actorId}/sessions/{sessionId}` に保存 - 完全な会話トレース
- **リフレクション**: `debugging/{actorId}` に保存 - 複数のエピソードからの合成知識

### エピソディックメモリを使用するタイミング

エピソディックメモリは以下の場合に使用します:
1. **順序的コンテキストが重要な場合**: アクションの順序とその結果が重要（例: デバッグワークフロー、トラブルシューティング手順）
2. **経験から学習する場合**: 過去の成功と失敗を分析してエージェントを改善したい
3. **プロセスの取得**: ユーザーが「前回 X をどう解決したか?」や「正確な手順を見せて」を思い出す必要がある

### チュートリアルの詳細

| 項目 | 詳細 |
|:-----|:-----|
| チュートリアルタイプ | リフレクション付きエピソディックメモリ |
| エージェントタイプ | デバッグアシスタント |
| フレームワーク | Strands Agents |
| LLM モデル | Claude Haiku 4.5 |
| メモリ戦略 | リフレクション設定付きエピソディックメモリ |
| 複雑さ | 中級 |

## 前提条件

- Python 3.10+
- AgentCore Memory 権限を持つ AWS 認証情報
- AgentCore サービスへのアクセス

## ステップ 1: 依存関係のインストールとセットアップ

In [ ]:
%pip install -qr requirements.txt

In [ ]:
import json
import logging
import sys
import uuid
from datetime import datetime, timezone
from typing import List, Dict, Any
from pprint import pprint

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("debugging-assistant")

# Import boto3 for control plane and data plane operations
import boto3

# Import Strands Agent framework
from strands import Agent, tool

logger.info("✅ すべての依存関係が正常にインポートされました")

In [ ]:
import os
# Configuration
REGION = os.getenv('AWS_REGION', 'us-west-2')
# Session identifiers
ACTOR_ID = "developer"

logger.info(f"リージョン {REGION} の設定が完了しました")
logger.info(f"アクターID: {ACTOR_ID}")

## ステップ 2: エピソディック戦略を使用したメモリの作成

**リフレクション設定**を含む**エピソディックメモリ戦略**で構成されたメモリリソースを作成します。これにより:
- 完全なデバッグセッションエピソードの保存
- 複数のエピソードからのリフレクションインサイトの自動生成

In [ ]:
# Initialize boto3 client for control and data plane operations
client = boto3.client(
    'bedrock-agentcore',
    region_name=REGION,
)
memory_client = boto3.client(
  'bedrock-agentcore-control',
   region_name=REGION,
)

In [ ]:
# Define episodic memory strategy with reflections as dictionary
memory_name = "DebugAssistantEpisodic"

# Episodic memory is implemented as a customMemoryStrategy
episodic_strategy = {
    "episodicMemoryStrategy": {
      "name": "DebuggingEpisodeExtractor",
      "description": "Creates debugging session episodes with reflections per actor",
      "namespaces": [
        "debugging/{actorId}/sessions/{sessionId}"
      ],
      "reflectionConfiguration": {
        "namespaces": [
          "debugging/{actorId}" # should be an exact prefix of the episodic memory namespace.
        ]
      }
    }
}
logger.info(f"戦略を設定: {episodic_strategy['episodicMemoryStrategy']['name']}")
logger.info(f"エピソードネームスペース: {episodic_strategy['episodicMemoryStrategy']['namespaces'][0]}")
logger.info(f"リフレクションネームスペース: {episodic_strategy['episodicMemoryStrategy']['reflectionConfiguration']['namespaces'][0]}")

In [ ]:
# Get or create memory
try:
    # Try to find existing memory first
    list_response = memory_client.list_memories(maxResults=100)
    memory_id = None
    for mem in list_response.get('memories', []):
        detail = memory_client.get_memory(memoryId=mem['id'])
        if detail['memory'].get('name') == memory_name:
            memory_id = mem['id']
            logger.info(f"✅ 既存のメモリを使用: {memory_id}")
            break
    
    # Create if not found
    if not memory_id:
        logger.info(f"新しいメモリを作成中: {memory_name}")
        response = memory_client.create_memory(
            name=memory_name,
            description="Episodic memory for debugging assistant with reflections",
            eventExpiryDuration=90,
            memoryStrategies=[episodic_strategy],
            clientToken=str(uuid.uuid4())
        )
        memory_id = response['memory']['id']
        logger.info(f"✅ メモリが作成されました: {memory_id}")
        
        # Wait for ACTIVE
        import time
        for _ in range(30):
            status = memory_client.get_memory(memoryId=memory_id)['memory']['status']
            if status == 'ACTIVE':
                logger.info("✅ メモリがアクティブになりました")
                break
            time.sleep(10)
            
except Exception as e:
    logger.error(f"❌ メモリの取得/作成に失敗: {e}")
    raise

## ステップ 4: 履歴デバッグセッションでメモリをハイドレート

過去のデバッグセッションをエピソディックメモリにロードしましょう。各セッションは完全なデバッグワークフローを表します。

In [ ]:
import os
import glob

# Load all session data files
data_dir = "./data"
session_files = sorted(glob.glob(f"{data_dir}/*.json"))

logger.info(f"ハイドレート対象のセッションファイルが {len(session_files)} 件見つかりました")

# Hydrate each session
for session_file in session_files:
    session_name = os.path.basename(session_file).replace('.json', '')
    session_id = f"{session_name}_{datetime.now().strftime('%Y%m%d%H%M%S')}"
    
    logger.info(f"セッションをハイドレート中: {session_name}")
    
    # Load conversation data
    with open(session_file, 'r') as f:
        conversation = json.load(f)
    
    # Convert to payload format
    payload = []
    for turn in conversation:
        conv_data = turn['conversational']
        payload.append({
            'conversational': {
                'content': {'text': conv_data['content']['text']},
                'role': conv_data['role']
            }
        })
    
    # Create event using boto3 directly
    event_timestamp = datetime.now(timezone.utc)
    result = client.create_event(
        memoryId=memory_id,
        actorId=ACTOR_ID,
        sessionId=session_id,
        eventTimestamp=event_timestamp,
        payload=payload
    )

    logger.info(f"   ✓ {len(payload)} ターンを保存 - イベントID: {result['event']['eventId']}")

logger.info(f"✅ {len(session_files)} 件のデバッグセッションのハイドレートが完了しました")

In [ ]:
### list memory records to see if its been extracted to LTM
import time
import pprint
reflection_namespace = f"debugging/{ACTOR_ID}"
# time.sleep(60)
# Use boto3 client directly to retrieve memory records
response = client.list_memory_records(
    memoryId=memory_id,
    namespace=reflection_namespace,
    maxResults=20
)
memories = response.get('memoryRecordSummaries', [])
logger.info(f"   {len(memories)} 件のメモリが見つかりました")
if memories:
    pprint.pp(json.loads(memories[0]["content"]["text"]))

In [ ]:
# check if reflections and episodes have been generated or not.
import pprint
# Use boto3 client directly to retrieve memory records
response = client.retrieve_memory_records(
memoryId=memory_id,
namespace=f"debugging/{ACTOR_ID}",
searchCriteria={
    'searchQuery': "memory leaks",
    "metadataFilters":[{"left": {"metadataKey": "x-amz-agentcore-memory-recordType"},
        "operator": "EQUALS_TO",
        "right": {"metadataValue": {"stringValue": "REFLECTION"}}
        }],          
    'topK': 10
},
    maxResults=20
)

reflections = response.get('memoryRecordSummaries', [])
logger.info(f"   {len(reflections)} 件の関連リフレクションが見つかりました")
if reflections:
    for reflection in reflections:
        reflection_json = json.loads(reflection['content']['text'])
        pprint.pp(reflection_json)

## ステップ 5: メモリ取得ツールの作成

エージェント用に 2 つの特化ツールを作成します:
1. **retrieve_process**: 詳細なステップバイステップのプロセス用に完全なエピソードトレースを取得
2. **retrieve_reflection_knowledge**: 複数のエピソードからの合成インサイトとパターンを取得

In [ ]:
def count_tokens(text: str) -> int:
    """Approximate token count for a text string."""
    return len(text)


def linearize_episodes(episodes: List[Dict], include_steps: bool = True,
                       include_reflections: bool = True) -> str:
    """Linearize episode data into human-readable format."""
    if not episodes:
        return "No relevant episodes found."

    output = []
    for idx, episode in enumerate(episodes, 1):
        content = episode.get('content', {})
        
        # Parse JSON from text field
        if 'text' in content:
            try:
                episode_data = json.loads(content['text'])
            except json.JSONDecodeError:
                output.append(f"Episode {idx}: Unable to parse content\n")
                continue
        else:
            output.append(f"Episode {idx}: No content available\n")
            continue
        
        output.append(f"{'='*80}\nEpisode {idx}\n{'='*80}")
        output.append(f"**Situation:** {episode_data.get('situation', 'N/A')}")
        output.append(f"**Intent:** {episode_data.get('intent', 'N/A')}")
        output.append(f"**Assessment:** {episode_data.get('assessment', 'N/A')}\n")
        
        if include_steps:
            turns = episode_data.get('turns', [])
            if turns:
                output.append("**Debugging Steps:**")
                for turn_idx, turn in enumerate(turns, 1):
                    output.append(f"\nStep {turn_idx}:")
                    output.append(f"  Situation: {turn.get('situation', 'N/A')}")
                    output.append(f"  Action: {turn.get('action', 'N/A')}")
                    output.append(f"  Thought: {turn.get('thought', 'N/A')}")
        
        if include_reflections:
            reflection = episode_data.get('reflection')
            if reflection:
                output.append(f"\n**Reflection:** {reflection}\n")
    
    result = "\n".join(output)
    logger.info(f"   エピソードトークン数: {count_tokens(result)}")
    return result


def linearize_reflections(reflections: List[Dict]) -> str:
    """Linearize reflection knowledge into human-readable format."""
    if not reflections:
        return "No reflection knowledge found."

    output = []
    for idx, reflection in enumerate(reflections, 1):
        content = reflection.get('content', {})
        score = reflection.get('score', 0)
        
        # Parse JSON from text field
        if 'text' in content:
            try:
                reflection_data = json.loads(content['text'])
            except json.JSONDecodeError:
                continue
        else:
            continue

        output.append(f"{'='*80}\nReflection {idx} (Relevance: {score:.2f})\n{'='*80}")
        output.append(f"**Title:** {reflection_data.get('title', 'Untitled')}")
        output.append(f"**Use Cases:** {reflection_data.get('use_cases', 'N/A')}")
        output.append(f"**Hints:** {reflection_data.get('hints', 'N/A')}\n")
    
    result = "\n".join(output)
    logger.info(f"   リフレクショントークン数: {count_tokens(result)}")
    return result


logger.info("✅ 線形化ヘルパー関数が作成されました")

In [ ]:
# Create memory retrieval tools for the agent

@tool
def retrieve_process(task: str, include_steps: bool = True) -> str:
    """
    Retrieve example processes to help solve the given task. Returns complete debugging
    episodes with configurable detail level.
    
    Use include_steps parameter to control verbosity:
    - Set include_steps=True when user asks for "exact steps", "full details", "how did we",
      "what steps did we take", or needs complete procedural information
    - Set include_steps=False for pattern/best practice queries where high-level context
      (situation, intent, assessment) is sufficient without step-by-step details

    Args:
        task: The task to solve that requires example processes
        include_steps: Whether to include detailed step-by-step turns (default: True)

    Returns:
        Formatted debugging episodes with optional detailed steps
    """
    logger.info(f"🔍 タスクのプロセスを取得中: {task} (include_steps={include_steps})")
    
    try:
        # Search in episode namespace
        namespace = f"debugging/{ACTOR_ID}/sessions/{session_id}"
        
        # Use boto3 client directly to retrieve memory records
        response = client.retrieve_memory_records(
            memoryId=memory_id,
            namespace=namespace,
            searchCriteria={
                'searchQuery': task,
                'topK': 3
            },
            maxResults=20
        )
        
        episodes = response.get('memoryRecordSummaries', [])
        logger.info(f"   {len(episodes)} 件の関連エピソードが見つかりました")
        
        # Linearize with configurable detail level
        return linearize_episodes(episodes, include_steps=include_steps, include_reflections=True)
        
    except Exception as e:
        logger.error(f"プロセス取得エラー: {e}")
        return f"Error retrieving processes: {str(e)}"


@tool
def retrieve_reflection_knowledge(task: str, k: int = 5) -> str:
    """
    Retrieve synthesized reflection knowledge from past agent experiences. Each knowledge 
    entry contains: (1) a descriptive title, (2) specific use cases for when to apply it, 
    and (3) actionable hints including best practices from successful episodes and common 
    pitfalls to avoid from failed episodes. Use this to get strategic guidance and patterns
    for similar tasks.

    Args:
        task: The current task to get strategic guidance for
        k: Number of reflection entries to retrieve (default: 5)

    Returns:
        Synthesized reflection knowledge from past debugging experiences
    """
    logger.info(f"🔍 タスクのリフレクション知識を取得中: {task}")
    
    try:
        # Search in reflection namespace
        namespace = f"debugging/{ACTOR_ID}"
        
        # Use boto3 client directly to retrieve memory records
        response = client.retrieve_memory_records(
            memoryId=memory_id,
            namespace=namespace,
            searchCriteria={
                'searchQuery': "memory leaks",
                "metadataFilters":[{"left":{"metadataKey": "x-amz-agentcore-memory-recordType"},
                                    "operator": "EQUALS_TO",
                                    "right": {"metadataValue": {"stringValue": "REFLECTION"}}
                                    }],          
                'topK': k
            },
            maxResults=20
        )
        
        reflections = response.get('memoryRecordSummaries', [])
        logger.info(f"   {len(reflections)} 件の関連リフレクションインサイトが見つかりました")
        
        # Linearize reflections
        return linearize_reflections(reflections)
        
    except Exception as e:
        logger.error(f"リフレクション取得エラー: {e}")
        return f"Error retrieving reflections: {str(e)}"


logger.info("✅ メモリ取得ツールが作成されました")

## ステップ 6: デバッグアシスタントエージェントの作成

メモリ取得ツールを装備した Strands エージェントを作成します。

In [ ]:
# Create the debugging assistant agent
debugging_agent = Agent(
    model="global.anthropic.claude-haiku-4-5-20251001-v1:0",
    tools=[retrieve_process, retrieve_reflection_knowledge],
    system_prompt="""You are an expert Debugging Assistant with access to episodic memory.

Your capabilities:
- Retrieve past debugging episodes with complete step-by-step processes
- Access synthesized reflection knowledge showing patterns and best practices
- Provide guidance based on successful debugging experiences
- Warn about common pitfalls observed in past failures

When helping users:
1. Use retrieve_reflection_knowledge for strategic guidance, patterns, and high-level advice
2. Use retrieve_process when users need exact steps or want to recall what was done in a specific session
3. Synthesize insights from memory with your own reasoning
4. Be specific and actionable in your recommendations

Always explain your reasoning and cite relevant past experiences when available."""
)

logger.info("✅ デバッグアシスタントエージェントが作成されました")

## ステップ 7: デバッグアシスタントのテスト

エージェントがエピソディックメモリとリフレクションをどのように使用するかを確認するために、さまざまなシナリオをテストしましょう。

### テスト 1: 戦略的ガイダンスのクエリ（リフレクション知識）

In [ ]:
# Test 1: Get strategic guidance for memory issues
query1 = "My application is running out of memory when processing large datasets. What should I look for?"

logger.info(f"\n{'='*80}")
logger.info(f"テスト 1: メモリ問題のガイダンス")
logger.info(f"{'='*80}")
logger.info(f"クエリ: {query1}\n")

response1 = debugging_agent(query1)

print("\n" + "="*80)
print("エージェントの応答:")
print("="*80)
print(response1)

### テスト 2: 具体的なプロセスの詳細のクエリ

In [ ]:
# Test 2: Get specific debugging process
query2 = "Show me the exact steps for debugging a timeout issue with external API calls."

logger.info(f"\n{'='*80}")
logger.info(f"テスト 2: API タイムアウトデバッグプロセス")
logger.info(f"{'='*80}")
logger.info(f"クエリ: {query2}\n")

response2 = debugging_agent(query2)

print("\n" + "="*80)
print("エージェントの応答:")
print("="*80)
print(response2)

### テスト 3: パターン認識のクエリ

In [ ]:
# Test 3: Get patterns and best practices for concurrency issues
query3 = "What are common patterns and best practices for handling race conditions in multi-threaded applications?"

logger.info(f"\n{'='*80}")
logger.info(f"テスト 3: レースコンディションのパターン")
logger.info(f"{'='*80}")
logger.info(f"クエリ: {query3}\n")

response3 = debugging_agent(query3)

print("\n" + "="*80)
print("エージェントの応答:")
print("="*80)
print(response3)

### テスト 4: 特定のセッションの呼び出し

In [ ]:
# Test 4: Recall what was done in memory leak session
query4 = "What debugging steps did we take when we encountered the memory leak issue? I need the full details."

logger.info(f"\n{'='*80}")
logger.info(f"テスト 4: メモリリークセッションの呼び出し")
logger.info(f"{'='*80}")
logger.info(f"クエリ: {query4}\n")

response4 = debugging_agent(query4)

print("\n" + "="*80)
print("エージェントの応答:")
print("="*80)
print(response4)

## ステップ 8: 直接メモリ検査

エピソディックメモリとリフレクションに保存されている内容を直接検査しましょう。

In [ ]:
# Inspect episodes directly using boto3
logger.info("" + "="*80)
logger.info("エピソードの直接検査")
logger.info("="*80)

# Retrieve episodes using boto3 directly
namespace = f"debugging/{ACTOR_ID}/sessions"
response = client.retrieve_memory_records(
    memoryId=memory_id,
    namespace=namespace,
    searchCriteria={
        'searchQuery': 'debugging',
        'topK': 2
    },
    maxResults=10
)

episodes = response.get('memoryRecordSummaries', [])

print(f"メモリ内に {len(episodes)} 件のエピソードが見つかりました:")
for idx, episode in enumerate(episodes, 1):
    print(f"エピソード {idx}:")
    pprint.pp(episode, depth=2, width=100)
    print("-" * 80)

In [ ]:
import pprint
response = client.retrieve_memory_records(
memoryId=memory_id,
namespace=reflection_namespace,
searchCriteria={
    'searchQuery': "memory leaks",
    "metadataFilters":[{
        "left":{"metadataKey": "x-amz-agentcore-memory-recordType"},
        "operator": "EQUALS_TO",
        "right": {"metadataValue": {"stringValue": "REFLECTION"}}
                        }],          
    'topK': 10
},
    maxResults=20
)

reflections = response.get('memoryRecordSummaries', [])
logger.info(f"   {len(reflections)} 件の関連リフレクションが見つかりました")
if reflections:
    for reflection in reflections:
        reflection_json = json.loads(reflection['content']['text'])
        pprint.pp(reflection_json)

## まとめ

### 達成したこと

- boto3 を使用してリフレクション設定付きのエピソディックメモリを作成
- 履歴デバッグセッションでメモリをハイドレート
- エピソードとリフレクション用の特化取得ツールを構築
- Strands フレームワークを使用してインテリジェントなデバッグアシスタントを作成
- 戦略的ガイダンス取得と詳細プロセス取得のデモンストレーション

### 主な学び

1. **エピソディックメモリ**は時間的コンテキストを持つ完全なやり取りシーケンスを保持
2. **リフレクション**は複数のエピソードからパターンとインサイトを自動的に合成
3. **線形化**は LLM 消費用に構造化データをフォーマットすることでコンテキストを最適化
4. **ツール選択**が重要: 戦略にはリフレクション、詳細手順にはエピソードを使用
5. **Boto3 直接アクセス**により Genesis Memory API 操作の完全な制御が可能

### このパターンを使用するタイミング

- 過去のチケット解決から学習する**テクニカルサポートシステム**
- 成功した診断手順を思い出す**トラブルシューティングアシスタント**
- 知識移転のために専門家のワークフローをキャプチャする**トレーニングシステム**
- 過去の結果分析がより良いプラクティスを推進する**プロセス改善**シナリオ

## クリーンアップ（オプション）

完了したらメモリリソースを削除するにはコメントを解除してください。

In [ ]:
# Uncomment to delete memory resource using boto3
# try:
#     client.delete_memory(memoryId=memory_id, clientToken=str(uuid.uuid4()))
#     logger.info(f"✅ Successfully deleted memory: {memory_id}")
# except Exception as e:
#     logger.error(f"Error deleting memory: {e}")